In [ ]:
import random
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torch.nn.functional import one_hot
from torch.utils.data import Dataset, TensorDataset, DataLoader, WeightedRandomSampler, SubsetRandomSampler, random_split, Subset
from torchvision.transforms import Compose, ToTensor, Normalize, ToPILImage, \
                                    RandomHorizontalFlip, Resize, transforms, RandomCrop, RandomRotation
from torchvision.datasets import ImageFolder

from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

PATH = "./data/dataset"
torch.manual_seed(13)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%run -i dataprep_v1.py

In [ ]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# dataset = ImageFolder(root=PATH, transform=transform)

In [ ]:
# labels = [label for _, label in dataset]
# one_hot_labels = one_hot(torch.tensor(labels), num_classes=len(dataset.classes)).float()

In [ ]:
# for i in range(len(dataset)):
    # dataset.samples[i] = (dataset.samples[i][0], one_hot_labels[i])

In [ ]:
# from sklearn.model_selection import train_test_split

# total_samples = len(dataset)
# indices = list(range(total_samples))
# train_indices, val_indices = train_test_split(indices, test_size=0.15, random_state=42)

In [ ]:
# train_sampler = Subset(dataset, train_indices)
# val_sampler = Subset(dataset, val_indices)

# train_loader = DataLoader(train_sampler, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_sampler, batch_size=32, shuffle=False)

In [ ]:
train_loader.dataset[0]

In [ ]:
#MODEL
from torchvision.models.resnet import resnet101, ResNet101_Weights, resnet152, ResNet152_Weights, resnet50, ResNet50_Weights

class ResnetCustom(nn.Module):
    def __init__(self, num_classes, p=.5) -> None:
        super(ResnetCustom, self).__init__()
        self.resnet = resnet101(weights=ResNet101_Weights.DEFAULT)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(p=p),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        x = self.resnet(x)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_classes = 151
resnet_v1 = ResnetCustom(num_classes, p=.6)
multi_loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(resnet_v1.resnet.fc.parameters(), lr=3e-4)
resnet_v1.to(device=device)

In [ ]:
# training loop
n_epochs = 60
train_losses = []
val_losses = []

for epoch in range(n_epochs):

    batch_losses = []
    batch_val_losses = []

    for x_batch, y_batch in train_loader:

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        resnet_v1.train()

        yhat = resnet_v1(x_batch)
        loss = multi_loss_fn(yhat, y_batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        batch_losses.append(loss.item())
    
    train_epoch_loss = np.mean(batch_losses)
    train_losses.append(train_epoch_loss)

    with torch.no_grad():

        for x_batch, y_batch in val_loader:

            resnet_v1.eval()

            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            yhat = resnet_v1(x_batch)
            loss = multi_loss_fn(yhat, y_batch)

            batch_val_losses.append(loss.item())

    val_epoch_loss = np.mean(batch_val_losses)
    val_losses.append(val_epoch_loss)

    print(f"Epoch {epoch+1}/{n_epochs}: loss = {train_epoch_loss}, val_loss = {val_epoch_loss}")
            
